# تدريب التعلم المعزز (RL) على موازنة عمود على عربة

هذا الدفتر هو جزء من [منهج الذكاء الاصطناعي للمبتدئين](http://aka.ms/ai-beginners). وقد استُلهم من [البرنامج التعليمي الرسمي لـ PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) و[تنفيذ Cartpole باستخدام PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

في هذا المثال، سنستخدم التعلم المعزز (RL) لتدريب نموذج على موازنة عمود على عربة يمكنها التحرك إلى اليسار واليمين على محور أفقي. سنستخدم بيئة [OpenAI Gym](https://www.gymlibrary.ml/) لمحاكاة العمود.

> **ملاحظة**: يمكنك تشغيل كود هذا الدرس محليًا (على سبيل المثال، من خلال Visual Studio Code)، وفي هذه الحالة ستُفتح المحاكاة في نافذة جديدة. عند تشغيل الكود عبر الإنترنت، قد تحتاج إلى إجراء بعض التعديلات على الكود، كما هو موضح [هنا](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

سنبدأ بالتأكد من تثبيت Gym:


In [ ]:
import sys
!{sys.executable} -m pip install gym

الآن دعونا ننشئ بيئة CartPole ونرى كيفية العمل عليها. تمتلك البيئة الخصائص التالية:

* **مجال الحركة** هو مجموعة الحركات الممكنة التي يمكننا تنفيذها في كل خطوة من خطوات المحاكاة  
* **مجال الملاحظات** هو مجال الملاحظات التي يمكننا القيام بها  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

دعونا نرى كيف تعمل المحاكاة. الحلقة التالية تشغل المحاكاة حتى تعيد `env.step` العلم الخاص بإنهاء العملية `done`. سنختار الإجراءات بشكل عشوائي باستخدام `env.action_space.sample()`، مما يعني أن التجربة ستفشل على الأرجح بسرعة كبيرة (بيئة CartPole تنتهي عندما تكون سرعة العربة، أو موقعها، أو زاويتها خارج حدود معينة).

> ستفتح المحاكاة في نافذة جديدة. يمكنك تشغيل الكود عدة مرات ومراقبة كيفية تصرفه.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

يمكنك ملاحظة أن الملاحظات تحتوي على 4 أرقام. وهي:
- موقع العربة
- سرعة العربة
- زاوية العصا
- معدل دوران العصا

`rew` هو المكافأة التي نحصل عليها في كل خطوة. يمكنك أن ترى أنه في بيئة CartPole يتم منحك نقطة واحدة لكل خطوة محاكاة، والهدف هو تعظيم إجمالي المكافآت، أي الوقت الذي تستطيع فيه CartPole الحفاظ على التوازن دون السقوط.

أثناء التعلم التعزيزي، هدفنا هو تدريب **سياسة** $\pi$، التي ستخبرنا لكل حالة $s$ بالإجراء $a$ الذي يجب اتخاذه، بمعنى آخر $a = \pi(s)$.

إذا كنت تريد حلاً احتماليًا، يمكنك اعتبار السياسة كإرجاع مجموعة من الاحتمالات لكل إجراء، أي أن $\pi(a|s)$ تعني احتمال أن نتخذ الإجراء $a$ في الحالة $s$.

## طريقة تدرج السياسة

في أبسط خوارزمية للتعلم التعزيزي، والتي تُسمى **تدرج السياسة**، سنقوم بتدريب شبكة عصبية لتوقع الإجراء التالي.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

سوف نقوم بتدريب الشبكة عن طريق إجراء العديد من التجارب وتحديث شبكتنا بعد كل تشغيل. دعنا نحدد وظيفة تقوم بتشغيل التجربة وتعيد النتائج (ما يسمى **التتبع**) - جميع الحالات، الإجراءات (واحتمالاتها الموصى بها)، والمكافآت:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

يمكنك تشغيل حلقة واحدة بشبكة غير مدربة وملاحظة أن إجمالي المكافأة (أي طول الحلقة) منخفض جدًا:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

أحد الجوانب الصعبة في خوارزمية تدرج السياسة هو استخدام **المكافآت المخصومة**. الفكرة هي أننا نحسب متجه المكافآت الإجمالية في كل خطوة من خطوات اللعبة، وخلال هذه العملية نقوم بخصم المكافآت المبكرة باستخدام معامل $gamma$. كما نقوم أيضًا بتطبيع المتجه الناتج، لأننا سنستخدمه كوزن للتأثير على تدريبنا:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

الآن دعونا نبدأ التدريب الفعلي! سنقوم بتشغيل 300 حلقة، وفي كل حلقة سنقوم بما يلي:

1. تشغيل التجربة وجمع المسار.
2. حساب الفرق (`gradients`) بين الإجراءات المتخذة والاحتمالات المتوقعة. كلما كان الفرق أقل، كلما كنا أكثر تأكدًا من أننا اتخذنا الإجراء الصحيح.
3. حساب المكافآت المخصومة وضرب التدرجات بالمكافآت المخصومة - هذا سيضمن أن الخطوات ذات المكافآت الأعلى سيكون لها تأثير أكبر على النتيجة النهائية مقارنة بالخطوات ذات المكافآت الأقل.
4. الإجراءات المستهدفة المتوقعة لشبكتنا العصبية سيتم أخذها جزئيًا من الاحتمالات المتوقعة أثناء التشغيل، وجزئيًا من التدرجات المحسوبة. سنستخدم معامل `alpha` لتحديد إلى أي مدى يتم أخذ التدرجات والمكافآت في الاعتبار - وهذا ما يسمى *معدل التعلم* لخوارزمية التعزيز.
5. وأخيرًا، نقوم بتدريب شبكتنا على الحالات والإجراءات المتوقعة، ونكرر العملية.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

الآن دعنا نشغل الحلقة مع العرض لنرى النتيجة:


In [ ]:
_ = run_episode(render=True)

نأمل أن تتمكن الآن من رؤية أن العمود يمكنه التوازن بشكل جيد!

## نموذج الممثل-الناقد

نموذج الممثل-الناقد هو تطوير إضافي لنموذج تدرجات السياسات، حيث نقوم ببناء شبكة عصبية لتعلم كل من السياسة والمكافآت المقدرة. تحتوي الشبكة على مخرجاتين (أو يمكنك اعتبارها شبكتين منفصلتين):
* **الممثل** سيقترح الإجراء الذي يجب اتخاذه من خلال إعطائنا توزيع احتمالات الحالة، كما هو الحال في نموذج تدرجات السياسات.
* **الناقد** سيقدر ما ستكون المكافأة الناتجة عن تلك الإجراءات. يقوم بإرجاع إجمالي المكافآت المقدرة في المستقبل عند الحالة المعطاة.

لنقم بتعريف مثل هذا النموذج:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

نحتاج إلى تعديل وظائف `discounted_rewards` و `run_episode` قليلاً:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


الآن سنقوم بتشغيل الحلقة الرئيسية للتدريب. سنستخدم عملية تدريب الشبكة اليدوية من خلال حساب دوال الخسارة المناسبة وتحديث معلمات الشبكة:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## الخلاصة

لقد رأينا في هذا العرض التوضيحي خوارزميتين للتعلم المعزز: خوارزمية بسيطة تعتمد على تدرج السياسة، وخوارزمية أكثر تطورًا تعتمد على الممثل-الناقد. يمكنك ملاحظة أن هذه الخوارزميات تعمل بمفاهيم مجردة مثل الحالة، الإجراء والمكافأة - مما يعني أنه يمكن تطبيقها على بيئات مختلفة تمامًا.

يتيح لنا التعلم المعزز اكتشاف أفضل استراتيجية لحل المشكلة بمجرد النظر إلى المكافأة النهائية. حقيقة أننا لا نحتاج إلى مجموعات بيانات معنونة تتيح لنا تكرار المحاكاة عدة مرات لتحسين نماذجنا. ومع ذلك، لا تزال هناك العديد من التحديات في مجال التعلم المعزز، والتي يمكنك التعرف عليها إذا قررت التركيز أكثر على هذا المجال المثير للاهتمام في الذكاء الاصطناعي.



---

**إخلاء المسؤولية**:  
تم ترجمة هذه الوثيقة باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار الوثيقة الأصلية بلغتها الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
